In [1]:
!pip install holidays

In [2]:
import pandas as pd
import datetime
from sklearn.preprocessing import LabelEncoder
import holidays
import seaborn as sns
import matplotlib.pyplot as plt

train = pd.read_csv("C:/Users/DO YEON/Downloads/DACON 제주 특산물 가격예측/open/train.csv")
test = pd.read_csv("C:/Users/DO YEON/Downloads/DACON 제주 특산물 가격예측/open/train.csv")

#날짜 파생변수 생성
#날짜 데이터 이해 위해 TIMESTAMP 생성 년/월/요일/주 파생변수 생성
train["timestamp"] = pd.to_datetime(train["timestamp"])
test["timestamp"] = pd.to_datetime(test["timestamp"])
df = pd.concat([train,test]).reset_index(drop = True)

df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

#년/월/일 추가
df['year']=df['timestamp'].dt.year
df['month']=df['timestamp'].dt.month
df['day']=df['timestamp'].dt.day

#요일 추가
df['week_day']=df['timestamp'].dt.weekday

# 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
le = LabelEncoder()
df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

# 라벨 인코딩
df["year_month"] = le.fit_transform(df["year_month"])


# 주차 변수 추가
df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

# 주차 누적값
week_list=[]
for i in range(len(df['year'])) :
    if df['year'][i] == 2019 :
        week_list.append(int(df['week'][i]))
    elif df['year'][i] == 2020 :
        week_list.append(int(df['week'][i])+52)
    elif df['year'][i] == 2021 :
        week_list.append(int(df['week'][i])+52+53)
    elif df['year'][i] == 2022 :
        week_list.append(int(df['week'][i])+52+53+53)
    elif df['year'][i] == 2023 :
        week_list.append(int(df['week'][i])+52+53+53+52)
df['week_num']= week_list

In [3]:
import holidays

# 한국의 공휴일 정보를 가져오기
kr_holi = holidays.KR()

# 'timestamp' 열의 날짜가 휴일인지 여부를 반환하는 함수 정의
def make_holi(timestamp):
    if timestamp in kr_holi:
        return 1  # 휴일이면 1을 반환
    else:
        return 0  # 휴일이 아니면 0을 반환

# 'timestamp' 열을 기반으로 'holiday' 열 추가
df["holiday"] = df["timestamp"].map(lambda x: make_holi(x))


In [4]:
kr_holi = holidays.KR()

df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))
    
# train, test 분리하기
train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)

In [5]:
train

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,RD_A_S_20190101,2019-01-01,RD,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_C_S_20190101,2019-01-01,RD,C,S,0.0,0.0,2019,1,1,1,0,1,1,1
3,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,RD_D_S_20190101,2019-01-01,RD,D,S,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118789,CR_D_S_20230303,2023-03-03,CR,D,S,0.0,0.0,2023,3,3,4,50,9,219,0
118790,CR_E_J_20230303,2023-03-03,CR,E,J,54320.0,1777.0,2023,3,3,4,50,9,219,0
118791,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,50,9,219,0
118792,TG_E_S_20230303,2023-03-03,TG,E,S,26157.0,3791.0,2023,3,3,4,50,9,219,0


In [6]:
test

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday


In [7]:
# 각 품목별 극값을 제거 
# 각각의 아이템 유형("TG", "RD", "BC", "CB")에 대해 0이 아닌 가격들의 평균을 계산하고, 이 평균 값을 특정 인덱스에 할당
tg_idx = train[(train["item"]=="TG") & (train["price"]>20000)].index
rd_idx = train[(train["item"]=="RD") & (train["price"]>5000)].index
bc_idx = train[(train["item"]=="BC") & (train["price"]>8000)].index
cb_idx = train[(train["item"]=="CB") & (train["price"]>2300)].index

train.loc[tg_idx,"price"] = train[(train["item"]=="TG") & (train["price"]!=0)]["price"].mean()
train.loc[rd_idx,"price"] = train[(train["item"]=="RD") & (train["price"]!=0)]["price"].mean()
train.loc[bc_idx,"price"] = train[(train["item"]=="BC") & (train["price"]!=0)]["price"].mean()
train.loc[cb_idx,"price"] = train[(train["item"]=="CB") & (train["price"]!=0)]["price"].mean()
print(f"train의 컬럼 : {train.columns}")
print(f"test의 컬럼 : {test.columns}")


#감귤 컬럼 제외
train_notg = train[train["item"] !="TG"]
test_notg = test[test["item"] != "TG"]


#인코딩
#인덱스를 재설정하고, 이전 인덱스삭제
#'supply' 열을 삭제
#'item', 'corporation', 'location' 열에 대해 원-핫 인코딩을 수행
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])

# 'timestamp', 'supply', 'price' 열을 삭제
#'item', 'corporation', 'location' 열에 대해 원-핫 인코딩을 적용
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])



train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')


In [8]:
import numpy as np

# 공휴일이지만 안쉬는 날 제외하기

# 'item'이 "TG", 'holiday'가 1, 그리고 'price'가 0이 아닌 조건을 만족하는 모든 고유 'timestamp'를 찾고, 이 'timestamp'들에 해당하는 모든 행의 'holiday' 값을 0으로 변경
# 'item'이 "TG"이고, 'holiday' 값이 1이며, 'price'가 0이 아닌 행들을 필터링
#필터링된 데이터를 'timestamp' 기준으로 그룹화하고, 각 그룹의 개수를 계산
#그룹화된 데이터의 'timestamp' 열을 가져와 리스트로 변환하여 no_holi 변수에 저장
no_holi = list(train[(train["item"] =="TG") &(train["holiday"]==1) & (train["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])

#train 데이터프레임에서 no_holi 리스트에 있는 'timestamp' 값들을 가진 행 추출
#행들의 'holiday' 열에 해당하는 인덱스를 추출하여 noholi_idx 변수에 저장
noholi_idx = train[train["timestamp"].isin(no_holi)]["holiday"].index

#반복문 내에서 train.loc[idx, "holiday"] = 0은 해당 인덱스의 'holiday' 열 값을 0으로 설정
for idx in noholi_idx:
    train.loc[idx, "holiday"] = 0

# train 및 test 시간 순서로 정렬
train_tg = train[train["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)
test_tg = test[test["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_tg1.columns}")

#Xy 데이터프레임의 'price' 열에 대해 루트 적용
Xy["price"] = np.sqrt(Xy["price"])

train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday'],
      dtype='object')


In [9]:
#train 데이터프레임에서 'item' 열의 값이 "TG"인 모든 행들을 선택
train_tg2 = train[train["item"] =="TG"]
test_tg2 = test[test["item"] == "TG"]

Xy2 = pd.get_dummies(train_tg2.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg2 = pd.get_dummies(test_tg2.drop(columns = ["timestamp","supply","price","item"]), columns = [ "corporation","location"])
print(Xy2.columns)

#Xy2 데이터프레임의 'price' 열에 대해 루트 적용
Xy2["price"] = np.sqrt(Xy2["price"])

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


In [10]:
train

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,RD_A_S_20190101,2019-01-01,RD,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_C_S_20190101,2019-01-01,RD,C,S,0.0,0.0,2019,1,1,1,0,1,1,1
3,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,RD_D_S_20190101,2019-01-01,RD,D,S,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118789,CR_D_S_20230303,2023-03-03,CR,D,S,0.0,0.0,2023,3,3,4,50,9,219,0
118790,CR_E_J_20230303,2023-03-03,CR,E,J,54320.0,1777.0,2023,3,3,4,50,9,219,0
118791,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,50,9,219,0
118792,TG_E_S_20230303,2023-03-03,TG,E,S,26157.0,3791.0,2023,3,3,4,50,9,219,0


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 'train'이라는 DataFrame이 있다고 가정합니다. 실제 데이터에 맞게 수정하세요.
data = train

# X와 y를 정의합니다. 여기서는 'price' 열을 타겟 변수로 사용하는 것으로 가정합니다.
X = data.drop(columns=['price'])  # 'price' 열을 제외한 나머지 열들을 특성으로 사용합니다.
y = data['price']  # 'price' 열을 타겟 변수로 사용합니다.

# train_test_split을 사용하여 데이터를 훈련 세트와 테스트 세트로 나눕니다.
# 예시로 train_size=0.8로 설정하여 80%의 데이터만 사용합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
data=train

print("price:", data.columns)

price: Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')


In [18]:
df.reset_index()

,index,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,0,1,1,0
2,2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,0,1,1,0
3,3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,0,1,1,0
4,4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118789,118789,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0,49,9,219,0
118790,118790,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,1,49,9,219,0
118791,118791,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,2,50,9,219,1
118792,118792,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,3,50,9,219,0


In [21]:
df['col']=df.index

In [23]:
train


,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,RD_A_S_20190101,2019-01-01,RD,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_C_S_20190101,2019-01-01,RD,C,S,0.0,0.0,2019,1,1,1,0,1,1,1
3,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,RD_D_S_20190101,2019-01-01,RD,D,S,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118789,CR_D_S_20230303,2023-03-03,CR,D,S,0.0,0.0,2023,3,3,4,50,9,219,0
118790,CR_E_J_20230303,2023-03-03,CR,E,J,54320.0,1777.0,2023,3,3,4,50,9,219,0
118791,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,50,9,219,0
118792,TG_E_S_20230303,2023-03-03,TG,E,S,26157.0,3791.0,2023,3,3,4,50,9,219,0


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# 예시로 train_size=0.8로 설정하여 80%의 데이터만 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 'train'이라는 DataFrame이 있다고 가정합니다. 실제 데이터에 맞게 수정하세요.
data = train

# 문자열 데이터를 원-핫 인코딩으로 변환합니다.
X_encoded = pd.get_dummies(X, sparse=True)


# 데이터를 훈련 및 테스트 세트로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 예시: 정수형 데이터를 int32로 변환
X_train['price'] = X_train['price'].astype('int32')

# 특성 행렬을 표준화합니다.
scaler = StandardScaler()

# X_train을 2D 배열로 변환합니다.
X_train_scaled = scaler.fit_transform(X_train.values.reshape(-1, 1))
X_test_scaled = scaler.transform(X_test.values.reshape(-1, 1))

y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

# Ridge 회귀 모델을 생성합니다.
ridge = Ridge()

# alpha 값의 후보군을 지정합니다.
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# GridSearchCV를 사용하여 최적의 alpha 값을 찾습니다.
grid_search = GridSearchCV(ridge, param_grid, cv=5)
grid_search.fit(X_train_scaled, X_train_scaled['price'])

# 최적의 alpha 값을 출력합니다.
print("최적의 alpha 값:", grid_search.best_params_['alpha'])

# 훈련 세트 및 테스트 세트에서 R-squared 점수를 출력합니다.
print("훈련 R-squared:", grid_search.score(X_train_scaled, y_train))
print("테스트 R-squared:", grid_search.score(X_test_scaled, y_test))


KeyError: 'price'

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
#릿지훈련
ridge.fit(train_scaled, train_target)

print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))